In [179]:
import numpy as np
from numba import jit
import cupy as cp
from random import randint
from time import time
from itertools import permutations



In [180]:
def create_matrix(city_count):
    city_list = np.zeros(shape=(city_count,city_count))
    for row in range(city_count):
        for column in range(city_count):
            if column==row:
                continue
            elif column < row:
                city_list[row][column] = city_list[column][row]
            else:
                city_list[row][column] = randint(1,100)
    return city_list


In [181]:
jit(nopython=True)
def permutations_fast(lista):
    if len(lista) == 0:
        return []
    if len(lista)==1:
        return [lista]
    permList = []
    for i in range(len(lista)):
        m = lista[i]
        remLista = lista[:i] + lista[i+1:]
        for p in permutations_fast(remLista):
            permList.append([m] + p)
    
    return permList


In [182]:
jit(nopython=True)
def do_sum_fast(city_matrix,path):
    sum_of_path = 0
    starting_city = path[0]
    path_lenght = np.zeros(len(path))
    for city in range(len(path)-1):
        path_lenght[city] = city_matrix[starting_city][path[city+1]]
        starting_city = path[city+1]

    path_lenght[-1] = city_matrix[starting_city][path[0]] 
    sum_of_path = np.sum(path_lenght)
    return sum_of_path

In [183]:
jit(nopython=True)
def tsp_fast(city_numbers,city_matrix):
    min_path = 999999999
    path = []
    for perm in permutations(city_numbers):
        sum_of_perm = do_sum_fast(city_matrix,perm)
        if sum_of_perm < min_path:
            min_path = sum_of_perm
            path = perm
    return (path,min_path)


In [185]:
cities = 11
city_numbers = np.zeros(cities,dtype=np.int32)
for i in range(cities):
    city_numbers[i] = i
    
city_matrix = create_matrix(cities)
for i in city_matrix:
    print(i)

TPB = 4
threadsperblock = (TPB,TPB)
blockspergrid_x = int(np.ceil(cities)/threadsperblock[0])
blockspergrid_y = int(np.ceil(cities)/threadsperblock[1])
blockspergrid = (blockspergrid_x,blockspergrid_y)

print(blockspergrid)

print(tsp_fast(city_numbers,city_matrix))

start = time()
x = tsp_fast(city_numbers,city_matrix)
end = time()
print("Elapsed (after compilation) = %s" % (end - start))
print(x[0],x[1])


[ 0. 37. 73. 46. 76. 62. 22. 30. 30. 88.]
[37.  0. 42. 83.  4. 87. 98.  8. 64. 59.]
[73. 42.  0.  5. 47. 12. 23. 53. 47. 81.]
[46. 83.  5.  0. 93. 23. 67. 59. 73. 65.]
[76.  4. 47. 93.  0. 35. 90. 29. 17. 91.]
[62. 87. 12. 23. 35.  0. 49. 84.  8. 58.]
[22. 98. 23. 67. 90. 49.  0. 38. 65. 13.]
[30.  8. 53. 59. 29. 84. 38.  0. 83. 46.]
[30. 64. 47. 73. 17.  8. 65. 83.  0. 59.]
[88. 59. 81. 65. 91. 58. 13. 46. 59.  0.]
(2, 2)
((0, 3, 2, 5, 8, 4, 1, 7, 9, 6), 181.0)
Elapsed (after compilation) = 30.775622367858887
(0, 3, 2, 5, 8, 4, 1, 7, 9, 6) 181.0
